In [15]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove



R1 Raised always equals to the sum of deposits.

$
\varphi_{\mathbf{R} 1} \equiv \operatorname{sum}(\text{deposits)}= \text {Escrow.balance}
$

R2 The escrow never allows the beneficiary to withdraw the investments and the investors to claim refunds.

$
\varphi_{\mathbf{R} \mathbf{2}} \equiv \square(\neg(\text {withdraw()} \wedge \text {claimRefund()}))
$


In [16]:
# The smart contract as a transition system.
ts = Ts('Crowdsale')

GOAL = 10000
CLOSETIME = 10000
timestamp = z3.BitVec('now', 256)

addrSort = z3.BitVecSort(256)

state, stateOut = ts.add_var(z3.BitVecSort(2), name='state')
OPEN = 0
SUCCESS = 1
REFUND = 2

deposits, depositsOut = ts.add_var(z3.ArraySort(addrSort, z3.BitVecSort(256)),
        name= 'deposits')

totalDeposits, totalDepositsOut = ts.add_var(z3.BitVecSort(256), name='totalDeposits')

raised, raisedOut = ts.add_var(z3.BitVecSort(256), name='raised')

aux_withdraw, aux_withdrawOut = ts.add_var(z3.BoolSort(), name='aux_withdraw')

aux_refund, aux_refundOut = ts.add_var(z3.BoolSort(), name='aux_refund')


func, funcOut = ts.add_var(z3.StringSort(), name='func')



In [17]:
p = z3.BitVec('p',256)

ts.Init = z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,
    totalDeposits == 0,
    aux_withdraw == False,
    aux_refund == False,
    state == OPEN,
    func == "init",
)

amount = z3.BitVec('amount',256)

tr_invest = z3.And(
    timestamp <= CLOSETIME,
    raised < GOAL,

    stateOut == state,
    raisedOut == raised + amount,
    depositsOut == z3.Update(deposits,p,deposits[p]+amount),
    totalDepositsOut == totalDeposits+amount,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "invest"
)

tr_close_success = z3.And(
    raised >= GOAL,

    depositsOut == deposits,
    raisedOut == raised,
    totalDepositsOut == totalDeposits,
    stateOut == SUCCESS,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "close_success"
)

tr_close_refund = z3.And(
    timestamp > CLOSETIME,
    raised < GOAL,

    stateOut == REFUND,
    depositsOut == deposits,
    raisedOut == raised,
    totalDepositsOut == totalDeposits,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "close_refund"
)

tr_claimrefund = z3.And(
    state == REFUND,
    raised < GOAL,

    depositsOut == z3.Update(deposits,p,0),
    totalDepositsOut == totalDeposits - deposits[p],
    raisedOut == raised - deposits[p],
    stateOut == state,

    aux_refundOut == True,
    aux_withdrawOut == aux_withdraw,
    funcOut == "claimrefund"
)

tr_withdraw = z3.And(
    state == SUCCESS,
    
    totalDepositsOut == 0,
    depositsOut == deposits,
    raisedOut == 0,
    stateOut == state,

    aux_withdrawOut == True,
    aux_refundOut == aux_refund,
    funcOut == "withdraw"
)

ts.Tr = z3.Or(tr_invest, tr_close_refund, tr_close_success, tr_claimrefund, tr_withdraw)

In [18]:

# raised always equals to the sum of deposits
r1 = raised == totalDeposits
s = prove_inductive(ts, r1)


Prove init => property.
proved
Prove property is inductive.
proved


In [19]:
# bmc
index = 0

def pure_name(name):
    if "|" in name:
        return name[:name.index('|')]
    else:
        return name

def fresh(round, s, name):
    # print("s = ",s)
    global index
    index += 1
    return z3.Const(name+"|r%d|i%d" % (round, index), s)

def zipp(xs, ys):
    # print("xs:", xs)
    # print("ys:", ys)
    return [p for p in zip(xs, ys)]

def bmc(init, trans, goal, fvs, xs, xns):
    s = z3.Solver()
    s.add(init)
    count = 0
    while True:
        print("iteration ", count)
        count += 1
        p = fresh(count, z3.BoolSort(), "P")
        s.add(z3.Implies(p, goal))
        if z3.sat == s.check(p):
            # print(s.model())
            return s.model()
        s.add(trans)
        ys = [fresh(count, x.sort(), pure_name(x.__str__())) for x in xs]
        nfvs = [fresh(count, x.sort(), pure_name(x.__str__())) for x in fvs]
        # print("before:", trans)
        trans = z3.substitute(trans, 
                           zipp(xns + xs + fvs, ys + xns + nfvs))
        # print("zip:", zipp(xns + xs + fvs, ys + xns + nfvs))
        # print("after:", trans)
        goal = z3.substitute(goal, zipp(xs, xns))
        xs, xns, fvs = xns, ys, nfvs

def extract_model(model, var=None):
    rd = {}
    maxrd = -1
    for v in model:
        name = v.__str__()
        if "|" in name:
            round = int(name[name.index("|r")+2:name.index("|i")]) + 1
            maxrd = max(maxrd, round)
            index = int(name[name.index("|i")+2:])
            pname = pure_name(name)
            if round not in rd.keys():
                rd[round] = {}
            rd[round][pname] = model[v]
            # print(round, index)
        else:
            if name[-1] == "'":
                if 1 not in rd.keys():
                    rd[1] = {}
                rd[1][name] = model[v]
            else:
                if 0 not in rd.keys():
                    rd[0] = {}
                rd[0][name] = model[v]
    # print(rd)
    for i in range(maxrd-1):
        print("round {i}:".format(i=i))
        if var == None:
            print(rd[i])
        else:
            if var in rd[i].keys():
                print(rd[i][var])
            else:
                print(rd[i][var+"'"])

In [20]:
r2 = z3.Not(z3.And(aux_withdraw, aux_refund))
s = prove_inductive(ts, r2)

Prove init => property.
proved
Prove property is inductive.
failed to prove:  sat
Implies(And(Not(And(aux_withdraw, aux_refund)),
            Or(And(now <= 10000,
                   raised < 10000,
                   state' == state,
                   raised' == raised + amount,
                   deposits' ==
                   Store(deposits, p, deposits[p] + amount),
                   totalDeposits' == totalDeposits + amount,
                   aux_refund' == aux_refund,
                   aux_withdraw' == aux_withdraw,
                   func' == "invest"),
               And(now > 10000,
                   raised < 10000,
                   state' == 2,
                   deposits' == deposits,
                   raised' == raised,
                   totalDeposits' == totalDeposits,
                   aux_refund' == aux_refund,
                   aux_withdraw' == aux_withdraw,
                   func' == "close_refund"),
               And(raised >= 10000,
                   dep

In [21]:
g = z3.Not(r2)
fvs = [timestamp, p, amount]
xs = [deposits, totalDeposits, raised, state, aux_withdraw, aux_refund, func]
xns = [depositsOut, totalDepositsOut, raisedOut, stateOut, aux_withdrawOut, aux_refundOut, funcOut]
model = bmc(ts.Init, ts.Tr, g, fvs, xs, xns)

extract_model(model, "func")
extract_model(model)

iteration  0
iteration  1
iteration  2
iteration  3


iteration  4
iteration  5
round 0:
"init"
round 1:
"invest"
round 2:
"close_success"
round 3:
"withdraw"
round 4:
"close_refund"
round 5:
"claimrefund"
round 0:
{'state': 0, 'aux_withdraw': False, 'totalDeposits': 0, 'raised': 0, 'aux_refund': False, 'deposits': K(BitVec(256), 0), 'amount': 28948132737270584649408991136343569060027996654852875558425328340023180787712, 'now': 57896044618658097711785492504343953926634992332820282019728792003956564829969, 'func': "init", 'p': 0}
round 1:
{"totalDeposits'": 28948132737270584649408991136343569060027996654852875558425328340023180787712, "deposits'": Store(K(BitVec(256), 0),
      0,
      28948132737270584649408991136343569060027996654852875558425328340023180787712), "func'": "invest", "aux_refund'": False, "aux_withdraw'": False, "raised'": 28948132737270584649408991136343569060027996654852875558425328340023180787712, "state'": 0}
round 2:
{'aux_refund': False, 'deposits': Store(K(BitVec(256), 0),
      0,
      2894813273727058464940899113